<a href="https://colab.research.google.com/github/tomasonjo/mcp-workshop/blob/main/02-mcp-cypher-extended.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade fastmcp langchain-mcp-adapters langgraph langchain-openai mcp-neo4j-cypher

In [2]:
import threading
import time
import logging
import sys
from io import StringIO
from fastmcp import FastMCP, Client
from fastmcp.client.transports import StdioTransport, StreamableHttpTransport

# Set root logger to CRITICAL
logging.getLogger().setLevel(logging.CRITICAL)

# Suppress all MCP/uvicorn/FastAPI related loggers
loggers_to_suppress = [
    "uvicorn",
    "uvicorn.access",
    "uvicorn.error",
    "fastmcp"
]

for logger_name in loggers_to_suppress:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.CRITICAL)
    logger.propagate = False

# Also disable the default stream handler
logging.disable(logging.WARNING)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key")

Enter your OpenAI API key··········


In [4]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model_name = 'gpt-5'
embed_model = "text-embedding-ada-002"

llm = ChatOpenAI(model=model_name)
# Initialize embeddings model
embeddings = OpenAIEmbeddings(model=embed_model)

# Create main server
mcp = FastMCP("Movies Server")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
# Run this if you're running it locally
"""
# Create stdio transport for Neo4j MCP server
transport = StdioTransport(
    command="uvx",
    args=["mcp-neo4j-cypher@0.5.1", "--namespace", "movies"],
    env={
        "NEO4J_URI": "neo4j+s://demo.neo4jlabs.com",
        "NEO4J_USERNAME": "recommendations",
        "NEO4J_PASSWORD": "recommendations",
        "NEO4J_DATABASE": "recommendations"
    }
)

# Create client with the transport
backend = Client(transport)
"""

'\n# Create stdio transport for Neo4j MCP server\ntransport = StdioTransport(\n    command="uvx",\n    args=["mcp-neo4j-cypher@0.5.1", "--namespace", "movies"],\n    env={\n        "NEO4J_URI": "neo4j+s://demo.neo4jlabs.com",\n        "NEO4J_USERNAME": "recommendations",\n        "NEO4J_PASSWORD": "recommendations",\n        "NEO4J_DATABASE": "recommendations"\n    }\n)\n\n# Create client with the transport\nbackend = Client(transport)\n'

In [6]:
# Run this if running in Google Colab
import threading
import subprocess
import time

def run_server():
    subprocess.run([
        "mcp-neo4j-cypher",
        "--server-port", "8001",
        "--db-url", "neo4j+s://demo.neo4jlabs.com",
        "--username", "recommendations",
        "--password", "recommendations",
        "--database", "recommendations",
        "--transport", "http",
        "--namespace", "movies"
    ])

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(5)

transport = StreamableHttpTransport(
    url="http://localhost:8001/mcp"
)

# Create client with the transport
backend = Client(transport)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
# Create and mount proxy (not async)
proxy = FastMCP.as_proxy(backend, name="Neo4jProxy")
mcp.mount(proxy, prefix="movies")

In [8]:
from typing import Annotated
from langchain_openai import OpenAIEmbeddings

@mcp.tool(
    name="find_movies_by_plot",
    description="Find movies by searching plot descriptions semantically. Use natural language queries like 'movies about time travel' or 'romantic comedies set in Paris'.",
)
async def find_movies_by_plot(
    query: Annotated[str, "Natural language description of the movie plot or theme to search for"]
) -> str:
    """Search movies using vector similarity on plot embeddings."""

    # Generate embedding for the query
    query_embedding = embeddings.embed_query(query)

    # Neo4j vector search Cypher query
    cypher = """
    CALL db.index.vector.queryNodes('moviePlotsEmbedding', $top_k, $query_embedding)
    YIELD node, score
    RETURN node.title AS title,
           node.plot AS plot,
           node.year AS year,
           node.imdbRating AS rating,
           score
    ORDER BY score DESC
    LIMIT $top_k
    """

    # Call the mounted Neo4j MCP server's read tool
    async with Client(mcp) as client:
        result = await client.call_tool(
            "movies_movies-read_neo4j_cypher",
            {
                "query": cypher,
                "params": {
                    "query_embedding": query_embedding,
                    "top_k": 5
                }
            }
        )
    return result.content[0].text

In [9]:
@mcp.tool(
    name="movies_ontology",
    description="Get domain-specific terminology definitions for the Neo4j recommendations database to help translate user vocabulary into Cypher queries",

)
def get_recommendations_ontology() -> str:
    ontology = """
# Movie Domain Terminology for Text-to-Cypher

## User Vocabulary Definitions

### Movies

1. **Blockbuster**: A highly successful movie with imdbRating > 7.5 and revenue > 100000000

2. **Classic**: An older movie that's well-regarded, released before 1990 with imdbRating > 7.0

3. **Hidden Gem**: A lesser-known quality movie with imdbRating > 7.5 but revenue < 50000000

4. **Box Office Hit**: Movies with revenue > 200000000 regardless of rating

5. **Commercial Success**: Movies where revenue > (budget * 3), indicating strong profitability

6. **Profitable**: Movies where revenue > budget with positive return on investment

7. **Critically Acclaimed**: Movies with imdbRating > 8.0

8. **Box Office Bomb**: Movies where revenue < budget, indicating financial loss

9. **Sleeper Hit**: Movies with budget < 20000000 but revenue > 100000000

10. **Big Budget**: Movies with budget > 100000000

### Actors/People

11. **Bankable Star**: Person who has acted in multiple movies with revenue > 150000000

12. **Character Actor**: Person who has ACTED_IN relationships with 10+ different movies

13. **A-List Actor**: Person who has acted in multiple movies with imdbRating > 7.5 and high revenue

14. **Box Office Draw**: Person whose movies consistently generate revenue > 100000000

## Query Translation Notes
- "Best" typically means highest imdbRating
- "Most profitable" means highest (revenue - budget)
- "Highest grossing" means highest revenue
- "Best ROI" means highest (revenue / budget) ratio
- "Commercial failure" means revenue < budget
"""

    return ontology

In [10]:
# Run server silently in background thread
def run_server():
    mcp.run(transport="streamable-http", port=8002)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)

print("✓ Server ready with Neo4j tools mounted")
print("Server running at: http://localhost:8002")

╭──────────────────────────────────────────────────────────────────────────────╮                  
                 │                                                                              │                  
                 │                         ▄▀▀ ▄▀█ █▀▀ ▀█▀ █▀▄▀█ █▀▀ █▀█                        │                  
                 │                         █▀  █▀█ ▄▄█  █  █ ▀ █ █▄▄ █▀▀                        │                  
                 │                                                                              │                  
                 │                                FastMCP 2.13.1                                │                  
                 │                                                                              │                  
                 │                                                                              │                  
                 │                  🖥  Server name: Movies Server                               │                  
                 │                                                                              │                  
                 │                  📦 Transport:   HTTP                                        │                  
                 │                  🔗 Server URL:  http://127.0.0.1:8002/mcp                   │                  
                 │                                                                              │                  
                 │                  📚 Docs:        https://gofastmcp.com                       │                  
                 │                  🚀 Hosting:     https://fastmcp.cloud                       │                  
                 │                                                                              │                  
                 ╰──────────────────────────────────────────────────────────────────────────────╯

✓ Server ready with Neo4j tools mounted
Server running at: http://localhost:8002


In [11]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import os

async def query(q: str):
    client = MultiServerMCPClient({
        "neo4j-database": {
            "url": "http://localhost:8002/mcp",
            "transport": "streamable_http"
        }
    })

    tools = await client.get_tools()
    agent = create_react_agent(llm, tools)
    response = await agent.ainvoke({"messages": [{"role": "system", "content": """You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies.
Never ask follow-up questions regarding the style of response. Be concise. Don't complicate.
Consult ontology when ambigous terms are used by the user"""},
                                                 {"role": "user", "content": q}]})
    for message in response["messages"]:
        message.pretty_print()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [12]:
await query("Find movies where both aliens and kids appear?")

/tmp/ipython-input-1248004230.py:15: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. 
Never ask follow-up questions regarding the style of response. Be concise. Don't complicate.
Consult ontology when ambigous terms are used by the user
================================ Human Message =================================

Find movies where both aliens and kids appear?
================================== Ai Message ==================================
Tool Calls:
  movies_ontology (call_JGH54O22lKD164puPdTJTY7w)
 Call ID: call_JGH54O22lKD164puPdTJTY7w
  Args:
================================= Tool Message =================================
Name: movies_ontology


# Movie Domain Terminology for Text-to-Cypher

## User Vocabulary Definitions

### Movies

1. **Blockbuster**: A highly successful movie with imdbRating > 7.5 and revenu

In [14]:
await query("What are the latest hidden gem movies?")

================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies. 
Never ask follow-up questions regarding the style of response. Be concise. Don't complicate.
Consult ontology when ambigous terms are used by the user
================================ Human Message =================================

What are the latest hidden gem movies?
================================== Ai Message ==================================
Tool Calls:
  movies_ontology (call_PaUF4v5WLJvnRYzHLbGus4VM)
 Call ID: call_PaUF4v5WLJvnRYzHLbGus4VM
  Args:
================================= Tool Message =================================
Name: movies_ontology


# Movie Domain Terminology for Text-to-Cypher

## User Vocabulary Definitions

### Movies

1. **Blockbuster**: A highly successful movie with imdbRating > 7.5 and revenue > 1000